In [73]:
import pickle

import numpy as np
import pandas as pd
from xgboost import XGBRegressor, XGBClassifier

from module.mymodule import PipeLine, train_or_test, grid_search_cv
from features import pipe_1, pipe_9

In [48]:
df = pd.read_csv('./data/train.csv')

test_df = df[df.Cholesterol == 0]
train_df = df[df.Cholesterol != 0]
train_df = train_df.drop('HeartDisease', axis=1)

reg_pipe = PipeLine()
reg_pipe(train_df, 'Cholesterol')
reg_pipe.one_hot(reg_pipe.df_cat.columns)
pack = reg_pipe.fold_out_split(test_size=0.2)

In [50]:
from sklearn.model_selection import GridSearchCV
param_grid = {
                          'max_depth':[3, 5, 7, 9, 15],
                          'learning_rate': [0.05, 0.1, 0.3],
                          'n_estimators': [50, 75, 100, 150],
                          }

reg_model = GridSearchCV(estimator=XGBRegressor(random_state=42),
                        param_grid=param_grid,  # 設定した候補を代入
                        scoring=None,  # デフォルトではaccuracyを基準に探してくれる
                        refit=True,
                        cv=3,
                        n_jobs=-1)

eval_set = [(pack[1], pack[3])]  # x_train, x_test, y_train, y_test = pack
reg_model.fit(pack[0], pack[2], eval_set=eval_set, verbose=False)

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.3],
                         'max_depth': [3, 5, 7, 9, 15],
                         'n_estimators': [50, 75, 100, 150]})

#### 本番環境用

In [46]:
df = pd.read_csv('./data/train.csv')

pipe = PipeLine()
pipe(df)
pipe.one_hot(pipe.df_cat.columns)
pipe.df_num

pred_trg = pipe.df_num[pipe.df_num.Cholesterol==0]
pred_trg = pred_trg.drop('Cholesterol', axis=1)
pred = gs_model.predict(pred_trg)

pipe.df_num.loc[pipe.df_num.Cholesterol==0, 'Cholesterol'] = pred.astype('int')
pipe.df_num

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,230,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0
3,34,1,115,248,1,154,0,0.2,1,0,0,0,0,1,0,0,0,1
4,54,0,160,201,0,163,0,0.0,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,48,1,106,263,1,110,0,0.0,1,0,0,0,0,1,0,0,1,0
638,53,1,126,242,0,106,0,0.0,1,0,0,0,0,1,0,0,1,0
639,54,1,200,198,0,142,1,2.0,1,0,0,0,0,1,0,0,1,0
640,45,0,130,237,0,170,0,0.0,0,1,0,0,0,1,0,0,0,1


In [84]:
with open('./data/reg_model.pkl', 'wb') as f:
    pickle.dump(reg_model, f)

In [85]:
def regress_chol(df, train_flg, split_kwrg, retrain=False):
    pipe = PipeLine()
    pipe(df)
    pipe.one_hot(pipe.df_cat.columns)
    pipe.df_num
    
    with open('./data/reg_model.pkl', 'rb') as f:
        reg_model = pickle.load(f)
    
    pred_trg = pipe.df_num[pipe.df_num.Cholesterol==0]
    pred_trg = pred_trg.drop('Cholesterol', axis=1)
    pred = reg_model.predict(pred_trg)
    pipe.df_num.loc[pipe.df_num.Cholesterol==0, 'Cholesterol'] = pred.astype('int')
    display(pipe.df_num)

    pipe.standard_scaler()

    if retrain:
        return pipe.df_num, pipe.df_target
    pack = train_or_test(pipe, train_flg, split_kwrg)
    return pack

In [86]:
to_pipe ={
          'df': df,
          'split_kwrg': {'test_size': 0.2, 'to_array': True},
          'train_flg': True,
          'retrain': False,
            }
pipe_lines = [regress_chol, pipe_1, pipe_9]
data_set = {pipe.__name__: pipe(**to_pipe) for pipe in pipe_lines}

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,230,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0
3,34,1,115,248,1,154,0,0.2,1,0,0,0,0,1,0,0,0,1
4,54,0,160,201,0,163,0,0.0,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,48,1,106,263,1,110,0,0.0,1,0,0,0,0,1,0,0,1,0
638,53,1,126,242,0,106,0,0.0,1,0,0,0,0,1,0,0,1,0
639,54,1,200,198,0,142,1,2.0,1,0,0,0,0,1,0,0,1,0
640,45,0,130,237,0,170,0,0.0,0,1,0,0,0,1,0,0,0,1


                                Base(pipe_1)                               


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0


                          Onehot Standard(pipe_9)                          


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0


In [87]:
# XGBoost
xgboost = {'model': XGBClassifier,
           'param_grid': {
                          'max_depth':[3, 5, 7, 9, 15],
                          'learning_rate': [0.05, 0.1, 0.3],
                          'n_estimators': [50, 75, 100, 150],
                          },
           'model_arg': {'random_state': 42, 'early_stopping_rounds': 50}
          }

# モデルの候補を設定
model_candidates = [
                    xgboost,
                    #tree,
                    #svc
                    ]

trained_models = {}  # 訓練したモデルの格納先
for candidate in model_candidates:
    print(candidate['model'].__name__.center(50, '#'))
    models = {}
    for key, pack in data_set.items():
        print(key.center(50))
        models[key] = grid_search_cv(pack, **candidate)
    trained_models[candidate['model'].__name__] = models

##################XGBClassifier###################
                   regress_chol                   
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.914230,0.914591,0.927798,0.921147
test,0.868217,0.901235,0.890244,0.895706


                      pipe_1                      
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.871345,0.916996,0.837545,0.875472
test,0.806202,0.901408,0.780488,0.836601


                      pipe_9                      
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.941520,0.936396,0.956679,0.946429
test,0.868217,0.901235,0.890244,0.895706
